# Análise de Sentimentos

Treinando a machine learning, utilizando o dataset recebido por email. (Faça o download e coloque a pasta dentro do repositorio)

In [1]:
import os
import xmltodict
import pandas as pd
os.chdir('..')
from nlputils.lexical.normalizer import Normalizer

Pegando todos os arquivos xml do trainset

In [2]:
dataset_path = 'trainset/'
all_files = []
for directory in os.listdir(dataset_path):
    if directory != '.DS_Store':
        for stars in os.listdir(dataset_path + directory):
            for file in os.listdir(dataset_path + directory + '/' + stars):
                if 'xml' in file:
                    all_files.append('{}{}/{}/{}'.format(dataset_path, directory, stars, file))

Criando a estrutura que será utilizada pela Machine Learning

In [117]:
dataset = {'trainset':[], 'polarity':[], 'review':[]}
for idx, file in enumerate(all_files):
    filename = file.split('/')[-1]
    with open(file, 'r') as text_file:
        data = text_file.read()
        try:
            dict_data = xmltodict.parse(data)
            dataset['trainset'].append('train') if idx > (len(all_files)/2) else dataset['trainset'].append('test')
            polarity = float(dict_data['review']['stars']['@value'])
            dataset['polarity'].append(polarity)
            dataset['review'].append(str(dict_data['review']['opinion']))
        except:
            print('Arquivo {} está com xml mal formatado'.format(file))

In [118]:
dataframe = pd.DataFrame(data=dataset)
dataframe.head()

,polarity,review,trainset
0,3.0,Se não fosse o uniforme do palmeiras seria uma...,test
1,3.0,BBB\n\nO que gostei: GGG\n\nO que não gostei: ...,test
2,3.0,Um bom produto por um preço de melhor ainda.\n...,test
3,3.0,Essa camiseta do time esfecifico é muito boa p...,test
4,3.0,"NÃO TENHO MUiTO O QUE DiZER,AFiNAL COMPREi PAR...",test


Agora vamos ler todos os textos e adicionar um novo campo na dataframe que irá ter a sentença sem pontuações e stopwords

In [119]:
from nltk.corpus import stopwords
normalizer = Normalizer()

def preprocessing(text):
    text = normalizer.to_lowercase(text)
    text = normalizer.remove_ponctuations(text)
    tokens = normalizer.tokenize_words(text)
    tokens = normalizer.remove_stopwords(tokens)
    return ' '.join(tokens)

dataframe['normalized_review'] = dataframe['review'].apply(preprocessing)
dataframe.head()

,polarity,review,trainset,normalized_review
0,3.0,Se não fosse o uniforme do palmeiras seria uma...,test,uniforme palmeiras camisa bonita comprar cores...
1,3.0,BBB\n\nO que gostei: GGG\n\nO que não gostei: ...,test,bbb gostei ggg gostei gggg
2,3.0,Um bom produto por um preço de melhor ainda.\n...,test,bom produto preço melhor ainda gostei bom prod...
3,3.0,Essa camiseta do time esfecifico é muito boa p...,test,camiseta time esfecifico é boa causa material ...
4,3.0,"NÃO TENHO MUiTO O QUE DiZER,AFiNAL COMPREi PAR...",test,dizerafinal comprei presente


In [120]:
from sklearn.feature_extraction.text import TfidfVectorizer

train_reviews = dataframe[dataframe['trainset'] == 'train']['normalized_review'].values.tolist()
train_classes = dataframe[dataframe['trainset'] == 'train']['polarity'].values.tolist()
test_reviews = dataframe[dataframe['trainset'] == 'test']['normalized_review'].values.tolist()
test_classes = dataframe[dataframe['trainset'] == 'test']['polarity'].values.tolist()

transformer = TfidfVectorizer()
transformer.fit(train_reviews)
X = transformer.transform(train_reviews)
X_test = transformer.transform(test_reviews)

Treinando 

In [121]:
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [122]:
classifier = SVC()
classifier.fit(X, train_classes)

/home/bruno/Projetos/.virtualenvs/pln-ven/lib/python3.5/site-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto_deprecated',
  kernel='rbf', max_iter=-1, probability=False, random_state=None,
  shrinking=True, tol=0.001, verbose=False)

In [124]:
accuracy_score(test_classes, classifier.predict(X_test))

0.37370211242391693

In [125]:
classifier_lr = LogisticRegression()
classifier_lr.fit(X, train_classes)

/home/bruno/Projetos/.virtualenvs/pln-ven/lib/python3.5/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/home/bruno/Projetos/.virtualenvs/pln-ven/lib/python3.5/site-packages/sklearn/linear_model/logistic.py:460: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=None, solver='warn',
          tol=0.0001, verbose=0, warm_start=False)

In [126]:
accuracy_score(test_classes, classifier_lr.predict(X_test))

0.507906671440506

In [135]:
sentence = "Esse filme é fraco"
preprocessed_sentence = preprocessing(sentence)
print(preprocessed_sentence)
instance = transformer.transform([preprocessing(sentence)])
print(instance)
classifier_lr.predict(instance)

filme é fraco
  (0, 21978)	0.6504526192491843
  (0, 21348)	0.7595468320728326


array([3.])

In [136]:
sentence = "Esse filme é fraco"
preprocessed_sentence = preprocessing(sentence)
print(preprocessed_sentence)
instance = transformer.transform([preprocessing(sentence)])
print(instance)
classifier.predict(instance)

filme é fraco
  (0, 21978)	0.6504526192491843
  (0, 21348)	0.7595468320728326


array([5.])

In [137]:
import pickle

